---
# Transformer XL
---

---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../deep-learning-dna")

In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import math
import string

import Set_Transformer 
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utils as tfu

In [5]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=False)

---
# Load Data

In [6]:
#Import pretrained model
api = wandb.Api()
model_path = api.artifact("sirdavidludwig/deep-learning-dna/dnabert-pretrain-ablation-dim:8dim").download()
pretrained_model = dnabert.DnaBertModel.load(model_path)
pretrained_model

In [7]:
#Load datafiles
dataset_path = api.artifact("sirdavidludwig/nachusa-dna/dnasamples:v1").download('/data/dna_samples:v1')
samples = find_dbs(dataset_path + '/train')
samples[13]

wandb: Downloading large artifact dnasamples:v1, 4086.55MB. 1260 files... Done. 0:0:0.3


'/data/dna_samples:v1/train/WS-CCW-Jul2015_S82_L001_R1_001.db'

---
# Create Dataset

In [8]:
set_len = 400
sequence_len = 150
kmer = 3
batch_size = 20
batches_per_epoch = 20
augument = True
labels = DnaLabelType.SampleIds
seed = 0
rng = np.random.default_rng(seed)
random_samples = samples.copy()

In [9]:
dataset = DnaSampleGenerator(samples=samples[0:5], subsample_length = set_len, sequence_length=sequence_len,kmer=kmer,batch_size=batch_size,batches_per_epoch=batches_per_epoch,augment=augument,labels=labels, rng=rng)

In [10]:
rng.shuffle(random_samples)

In [11]:
random_samples[0:5]

['/data/dna_samples:v1/train/WS-MU-Oct2016_S71_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-HPN-Jul2016_S78_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes21-8-TCR_S22_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-CCE-Sep2015_S60_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wesley056-NegCtrl_S195_L001_R1_001.db']

--- 
# Batch Parameters

In [12]:
block_size = 200
seq_len = set_len
maxlen = set_len
vocab_size = 5
num_chars_data = set_len*sequence_len
max_files = len(dataset.samples)
max_files

5

In [13]:
if block_size-2 > seq_len:
    raise ValueError("Block size should not be bigger than sequence length")

In [14]:
print(maxlen)
print(vocab_size)
print(num_chars_data)

400
5
60000


---
# Create Embeddings

In [15]:
#Create 8 dimensional embeddings
pretrained_encoder = dnabert.DnaBertEncoderModel(pretrained_model.base)
pretrained_encoder.trainable = False

In [16]:
class Create_Embeddings(keras.layers.Layer):
    def __init__(self, encoder):
        super(Create_Embeddings, self).__init__()
        self.encoder = encoder
        
    def subbatch_predict(self, model, batch, subbatch_size, concat=lambda old, new: tf.concat((old, new), axis=0)):
        def predict(i, result=None):
            n = i + subbatch_size
            pred = tf.stop_gradient(model(batch[i:n]))
            if result is None:
                return [n, pred]
            return [n, concat(result, pred)]
        i, result = predict(0)
        batch_size = tf.shape(batch)[0]
        i, result = tf.while_loop(
            cond=lambda i, _: i < batch_size,
            body=predict,
            loop_vars=[i, result],
            parallel_iterations=1)

        return result
    
    def modify_data_for_input(self, data):
        batch_size = tf.shape(data)[0]
        subsample_size = tf.shape(data)[1]
        flat_data = tf.reshape(data, (batch_size*subsample_size, -1))
        encoded = self.subbatch_predict(self.encoder, flat_data, 128)
        return tf.reshape(encoded, (batch_size, subsample_size, -1))
    
    def call(self, data):
        return  self.modify_data_for_input(data)

---
# Setup

In [17]:
def clone_initializer(initializer):
    if isinstance(initializer, tf.keras.initializers.Initializer):
        return initializer.__class__.from_config(initializer.get_config())
    return initializer

In [18]:
def get_shape_list(tensor, expected_rank=None, name=None):
    if expected_rank is not None:
        assert_rank(tensor, expected_rank, name)

    shape = tensor.shape.as_list()

    non_static_indexes = []
    for (index, dim) in enumerate(shape):
        if dim is None:
            non_static_indexes.append(index)

    if not non_static_indexes:
        return shape

    dyn_shape = tf.shape(tensor)
    for index in non_static_indexes:
        shape[index] = dyn_shape[index]
    return shape

---
# Custom Loss and Accuracy

In [19]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def MaskedSparseCategoricalCrossentropy(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
def MaskedSparseCategoricalAccuracy(real, pred):
    accuracies = tf.equal(tf.cast(real,tf.int64), tf.argmax(pred, axis=2))
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)
    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

---
# Cache Memory

In [20]:
def _cache_memory(current_state, previous_state, memory_length, reuse_length=0):
    if memory_length is None or memory_length == 0:
        return None
    else:
        if reuse_length > 0:
            current_state = current_state[:, :reuse_length, :]

        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# XL Block

In [21]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 stack,
                 num_induce,
                 vocab_size,
                 hidden_size,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 norm_epsilon=1e-12,
                 inner_activation="relu",
                 kernel_initializer="variance_scaling",
                 inner_dropout=0.0,
                 **kwargs):

        super(TransformerXLBlock, self).__init__(**kwargs)
        self._stack = stack
        self._num_induce = num_induce
        self._vocab_size = vocab_size
        self._num_heads = num_attention_heads
        self._head_size = head_size
        self._hidden_size = hidden_size
        self._inner_size = inner_size
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._inner_activation = inner_activation
        self._norm_epsilon = norm_epsilon
        self._kernel_initializer = kernel_initializer
        self._inner_dropout = inner_dropout
        self._attention_layer_type = Set_Attention
        
    def build(self, input_shape):
        input_tensor = input_shape[0] if len(input_shape) == 2 else input_shape
        input_tensor_shape = tf.TensorShape(input_tensor)
        if len(input_tensor_shape.as_list()) != 3:
            raise ValueError("TransformerLayer expects a three-dimensional input of "
                                             "shape [batch, sequence, width].")
        batch_size, sequence_length, hidden_size = input_tensor_shape

        if len(input_shape) == 2:
            mask_tensor_shape = tf.TensorShape(input_shape[1])
            expected_mask_tensor_shape = tf.TensorShape(
                    [batch_size, sequence_length, sequence_length])
            if not expected_mask_tensor_shape.is_compatible_with(mask_tensor_shape):
                raise ValueError("When passing a mask tensor to TransformerXLBlock, "
                                                 "the mask tensor must be of shape [batch, "
                                                 "sequence_length, sequence_length] (here %s). Got a "
                                                 "mask tensor of shape %s." %
                                                 (expected_mask_tensor_shape, mask_tensor_shape))
        if hidden_size % self._num_heads != 0:
            raise ValueError(
                    "The input size (%d) is not a multiple of the number of attention "
                    "heads (%d)" % (hidden_size, self._num_heads))
            
            
        self._attention_layer= self._attention_layer_type(num_induce=self._num_induce, embed_dim=self._head_size,num_heads=self._num_heads, stack=self._stack,
                                                   use_layernorm=True,pre_layernorm=True,use_keras_mha=True)
        
        super(TransformerXLBlock, self).build(input_shape)

   
    def call(self,
             content_stream,
             content_attention_bias,
             positional_attention_bias,
             relative_position_encoding=None,
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        attention_output = {"content_attention": self._attention_layer(content_stream, state)}        

        return attention_output

---
# Transformer XL

In [22]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 stack, 
                 num_induce, 
                 vocab_size,
                 num_layers,
                 hidden_size,
                 maxlen,
                 embed_dim,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 initializer,
                 tie_attention_biases=True,
                 memory_length=None,
                 reuse_length=None,
                 inner_activation="relu",
                 **kwargs):
        super(TransformerXL, self).__init__(**kwargs)

        self._stack = stack
        self._num_induce = num_induce
        self._vocab_size = vocab_size
        self._initializer = initializer
        self._num_layers = num_layers
        self._hidden_size = hidden_size
        self._num_attention_heads = num_attention_heads
        self._head_size = head_size
        self._inner_size = inner_size
        self._inner_activation = inner_activation
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._tie_attention_biases = tie_attention_biases
        self._memory_length = memory_length
        self._reuse_length = reuse_length

        self.transformer_xl_layers = []
        
        for i in range(self._num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(
                            stack=self._stack,
                            num_induce=num_induce, 
                            vocab_size=self._vocab_size,
                            hidden_size=self._head_size * self._num_attention_heads,
                            num_attention_heads=self._num_attention_heads,
                            head_size=self._head_size,
                            inner_size=self._inner_size,
                            dropout_rate=self._dropout_rate,
                            attention_dropout_rate=self._attention_dropout_rate,
                            norm_epsilon=1e-12,
                            inner_activation=self._inner_activation,
                            kernel_initializer="variance_scaling",
                            name="layer_%d" % i))

        self.output_dropout = tf.keras.layers.Dropout(rate=self._dropout_rate)

    def call(self,
             content_stream,
             relative_position_encoding,
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        new_mems = []

        if state is None:
            state = [None] * self._num_layers
            
        for i in range(self._num_layers):
            new_mems.append(_cache_memory(content_stream, state[i], self._memory_length, self._reuse_length))
            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(
                    content_stream=content_stream,
                    content_attention_bias=None,
                    positional_attention_bias=None,
                    relative_position_encoding=None,
                    state=state[i],
                    content_attention_mask=content_attention_mask,
                    query_attention_mask=query_attention_mask,
                    target_mapping=target_mapping)
            
            content_stream = transformer_xl_output["content_attention"]

        output_stream = content_stream
        return output_stream, new_mems

In [23]:
class Set_Attention(keras.Model):
    def __init__(self, num_induce, embed_dim, num_heads, stack, use_layernorm, pre_layernorm, use_keras_mha):
        super(Set_Attention, self).__init__()
        
        self.set_attention = (Set_Transformer.InducedSetAttentionBlock(embed_dim=embed_dim,num_heads=num_heads, num_induce=num_induce, use_layernorm=use_layernorm,pre_layernorm=pre_layernorm,use_keras_mha=use_keras_mha))
      
    def call(self, data, mems):
                
            attention = self.set_attention([data, mems])
                
            return attention

---
# Xl Model Class

In [27]:
class XlModel(keras.Model):
    def __init__(self, stack, num_induce, max_files, encoder, block_size, seq_len_padded, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer):
        super(XlModel, self).__init__()
        
        self.block_size = block_size
        self.seq_len_padded = seq_len_padded
        self.embed_dim = embed_dim
        self.num_heads = num_attention_heads
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.maxlen = maxlen
        self.memory_length = memory_length
        self.max_files = max_files
        self.encoder = encoder
        self.isabs = []
        
        self.embedding_layer = Create_Embeddings(encoder)

        self.linear_layer = keras.layers.Dense(embed_dim)
        
        self.transformer_xl = TransformerXL(
                stack=stack, 
                num_induce=num_induce, 
                vocab_size=vocab_size,
                num_layers=num_layers,
                hidden_size=hidden_size,
                num_attention_heads=num_attention_heads,
                maxlen=maxlen,
                embed_dim=embed_dim,
                memory_length=memory_length,
                reuse_length=reuse_length,
                head_size=head_size,
                inner_size=inner_size,
                dropout_rate=dropout_rate,
                attention_dropout_rate=attention_dropout_rate,
                initializer=initializer, 
            )
        

        self.pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=1,embed_dim=embed_dim,num_heads=1,use_layernorm=True,pre_layernorm=True, use_keras_mha=True,is_final_block=True)
    
        self.reshape_layer = keras.layers.Reshape((embed_dim,))
   
        self.output_layer = keras.layers.Dense(self.max_files, activation=keras.activations.softmax)
        
    
    def call(self, x, training=None):        
 
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.memory_length, self.embed_dim))
        
        embeddings = self.embedding_layer(x)
            
        linear_transform = self.linear_layer(embeddings)    
            
        for i in range(0, self.seq_len_padded, self.block_size):
            block = embeddings[:,i:i+block_size]
            
            output, mems = self.transformer_xl(content_stream=block, relative_position_encoding=None, state=mems)
                
        pooling = self.pooling_layer(output)

        reshape = self.reshape_layer(pooling)

        output = self.output_layer(reshape)          
        
        return output

---
# Xl Parameters

In [28]:
#Xl Parameters
stack = 1
num_induce = 30
embed_dim = 8
num_layers = 10
hidden_size = 32
num_attention_heads = 8
memory_length = 200
reuse_length = 0
head_size = 8
inner_size = 32
dropout_rate = 0.01
attention_dropout_rate = 0.01
initializer = keras.initializers.RandomNormal(stddev=0.1) 

encoder = pretrained_encoder
epochs = 100

---
# Create Models

In [29]:
model = XlModel(stack, num_induce, max_files, encoder, block_size, seq_len, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer)
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = keras.optimizers.Nadam(1e-4), metrics = keras.metrics.SparseCategoricalAccuracy())
model.fit(dataset, epochs = epochs)

Epoch 1/100
20/20 [==============================] - 67s 2s/step - loss: 1.8455 - sparse_categorical_accuracy: 0.1925
Epoch 2/100
20/20 [==============================] - 38s 2s/step - loss: 1.7780 - sparse_categorical_accuracy: 0.1675
Epoch 3/100
20/20 [==============================] - 38s 2s/step - loss: 1.6693 - sparse_categorical_accuracy: 0.1750
Epoch 4/100
20/20 [==============================] - 38s 2s/step - loss: 1.6466 - sparse_categorical_accuracy: 0.1700
Epoch 5/100
20/20 [==============================] - 38s 2s/step - loss: 1.6363 - sparse_categorical_accuracy: 0.2500
Epoch 6/100
20/20 [==============================] - 38s 2s/step - loss: 1.6265 - sparse_categorical_accuracy: 0.1850
Epoch 7/100
20/20 [==============================] - 38s 2s/step - loss: 1.5994 - sparse_categorical_accuracy: 0.2150
Epoch 8/100
20/20 [==============================] - 38s 2s/step - loss: 1.5948 - sparse_categorical_accuracy: 0.1975
Epoch 9/100
20/20 [==============================] - 38s

KeyboardInterrupt: 

In [ ]:
u = model.evaluate(dataset[0][0], dataset[0][1])

In [ ]:
u = model.predict(dataset[0][0])

In [ ]:
u = u.argmax(-1)

In [46]:
u

array([0, 4, 2, 4, 0, 2, 4, 4, 0, 4, 0, 1, 1, 4, 0, 2, 2, 1, 2, 2])

In [42]:
dataset[0][1]

array([0, 0, 2, 3, 0, 4, 4, 4, 3, 3, 0, 1, 1, 3, 4, 3, 2, 1, 2, 2],
      dtype=int32)

In [49]:
samples[0:5]

['/data/dna_samples:v1/train/WS-AG-Apr2016_S85_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-Jul2015_S3_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-Jul2016_S31_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-May2015_S65_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-AG-Oct2016_S72_L001_R1_001.db']